In [1]:
import os, sys
from ZillowHouseData.logger import logger
from ZillowHouseData.exception import CustomException
import pandas as pd
import pickle

In [13]:
class DataPreprocessing:
    def __init__(self, file_name='ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv', start_date='2012-01-01'):
        #self.file_name = file_name
        #self.file_name = os.path.join('../','artifacts', 'data_ingestion',file_name)
        self.file_name = os.path.join('..','artifacts', 'data_ingestion',file_name)
        self.start_date = start_date

    def data_preprocessing(self):
        dtypes = {
            'indicator_id': 'object',
            'region_id': 'int32',
            'value': 'float32',
            'date': 'object'
        }
        df_list = []
        count = 0

        #raw_data_path: str=os.path.join('artifacts/data_ingestion/','ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv')
        print("\n printing os path current directory")
        print(os.getcwd())


        file_path = os.path.join('artifacts', 'data_ingestion', 'ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv')
        print("\n printing file existance")
        print(os.path.exists(file_path))

        
        for chunk in pd.read_csv(self.file_name, chunksize=1000000,
                                 usecols=['indicator_id', 'region_id', 'date', 'value'],
                                 dtype=dtypes,
                                 parse_dates=['date']):
            filtered_chunk = chunk[(chunk['date'] >= self.start_date)]
            df_list.append(filtered_chunk)

        trimmed_df = pd.concat(df_list, ignore_index=True)

        del df_list
        
        trimmed_data = pickle.dumps(trimmed_df)
    
        return trimmed_data

    def get_year_month(self, data):
        trimmed_df = pickle.loads(data)
        trimmed_df['year'] = pd.to_datetime(trimmed_df['date']).dt.year
        trimmed_df['month'] = pd.to_datetime(trimmed_df['date']).dt.month
        trimmed_data = pickle.dumps(trimmed_df)
        return trimmed_data

    def get_stats(self, data):
        trimmed_df = pickle.loads(data)
        intersted_indicators_stats = ['IRAM', 'CRAM', 'MRAM', 'LRAM', 'NRAM', 'SRAM']
        stat_df = trimmed_df[trimmed_df['indicator_id'].isin(intersted_indicators_stats)]
        stat_pivot_df = stat_df.pivot_table(index=['region_id', 'year', 'month'], columns='indicator_id',
                                           values='value', aggfunc='mean')
        stat_pivot_df.reset_index(inplace=True)
        stat_pivot_df.dropna(inplace=True)
        stat_pivot_df.to_csv("stat.csv")
        del stat_df
        trimmed_data = pickle.dumps(trimmed_df)
        return trimmed_data

    def get_merge(self, data):
        trimmed_df = pickle.loads(data)
        df = pd.read_csv('stat.csv')
        intersted_indicators_ZHVI = ['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z2BR', 'Z5BR', 'Z3BR', 'Z1BR', 'Z4BR']
        ZHVI_df = trimmed_df[trimmed_df['indicator_id'].isin(intersted_indicators_ZHVI)]
        del trimmed_df
        final_df = pd.merge(ZHVI_df, df, on=['region_id', 'year', 'month'], how='inner')
        final_data = pickle.dumps(final_df)
        final_df.to_csv("final.csv")
        return final_data


In [7]:
from ZillowHouseData.components.data_preprocessing import DataPreprocessing
from ZillowHouseData.logger import logger
from ZillowHouseData.exception import CustomException
from ZillowHouseData.config.configuration import ConfigurationManager
from ZillowHouseData.pipeline.stage_01_data_ingestion_pipeline import ingestion_stage
import sys

In [14]:
STAGE_NAME = "Data Preprocessing stage"

class DataPreprocessingTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
            # Create an instance of the DataPreprocessing class
            logger.info(f">>>>>> stage {STAGE_NAME} initated <<<<<<\n\nx==========x")
            data_preprocessor = DataPreprocessing()
            # Replace 'your_data_bytes_here' with the actual data bytes you want to process
            #data = b'your_data_bytes_here'
            # Stage 1: Data Preprocessing
            processed_data = data_preprocessor.data_preprocessing()
            # Stage 2: Extract Year and Month
            logger.info(f">>>>>> exact year & month <<<<<<\n\nx==========x")
            processed_data = data_preprocessor.get_year_month(processed_data)
            # Stage 3: Get Stats
            logger.info(f">>>>>> get stats <<<<<<\n\nx==========x")
            processed_data = data_preprocessor.get_stats(processed_data)
            # Stage 4: Merge Data
            logger.info(f">>>>>> merging data <<<<<<\n\nx==========x")
            final_data = data_preprocessor.get_merge(processed_data)
            print(final_data)



if __name__ == '__main__':
    #ingestion_stage()
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} initated <<<<<<\n\nx==========x")
        obj1 = DataPreprocessingTrainingPipeline()
        obj1.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise CustomException(e,sys)
    


[2023-11-10 20:32:28,322: INFO: 1457263588: >>>>>> stage Data Preprocessing stage initated <<<<<<

x==========x]
[2023-11-10 20:32:28,323: INFO: 1457263588: >>>>>> stage Data Preprocessing stage initated <<<<<<

x==========x]

 printing os path current directory
/Users/keshavkumarelankovan/Desktop/ML_OPS/Skirmish/Zillow-data-ML-project/notebook

 printing file existance
False
[2023-11-10 20:32:28,325: ERROR: 1457263588: [Errno 2] No such file or directory: '. /artifacts/data_ingestion/ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv']
Traceback (most recent call last):
  File "/var/folders/kp/6197l28d6yzb5wygdgr1hb7w0000gn/T/ipykernel_34285/1457263588.py", line 33, in <module>
    obj1.main()
  File "/var/folders/kp/6197l28d6yzb5wygdgr1hb7w0000gn/T/ipykernel_34285/1457263588.py", line 14, in main
    processed_data = data_preprocessor.data_preprocessing()
  File "/var/folders/kp/6197l28d6yzb5wygdgr1hb7w0000gn/T/ipykernel_34285/324563388.py", line 28, in data_preprocessing
    for chunk

CustomException: Error occured in python script name [/var/folders/kp/6197l28d6yzb5wygdgr1hb7w0000gn/T/ipykernel_34285/1457263588.py] line number [33] error message [[Errno 2] No such file or directory: '. /artifacts/data_ingestion/ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv']

In [23]:
file_name = 'ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv'
file = os.path.join('..','artifacts', 'data_ingestion',file_name)
print(os.path.exists(file))
print(os.getcwd())

True
/Users/keshavkumarelankovan/Desktop/ML_OPS/Skirmish/Zillow-data-ML-project/notebook


In [22]:
file_name = 'ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv'
#script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = os.path.dirname(os.path.realpath(sys.argv[0]))
print(script_dir)
file_relative_path = os.path.join(script_dir, '..','artifacts', 'data_ingestion', file_name)

# Check if the file exists
print("File exists:", os.path.exists(file_relative_path))

# Print the current working directory
print("Current working directory:", os.getcwd())


/Users/keshavkumarelankovan/Desktop/ML_OPS/Skirmish/Zillow-data-ML-project/mlops_zillow/lib/python3.10/site-packages
File exists: False
Current working directory: /Users/keshavkumarelankovan/Desktop/ML_OPS/Skirmish/Zillow-data-ML-project/notebook


In [25]:
#os.chdir('../')

In [3]:
# !pip install -U scikit-learn
# !pip install pandas

from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Create a LabelEncoder instance
# label_encoder = LabelEncoder()
df = pd.read_csv('/Users/hm/Documents/Github_Projects/Zillow-data-ML-project/artifacts/data_ingestion/final.csv')
# Fit and transform the 'indicator_id' column
# df['indicator_id'] = label_encoder.fit_transform(df['indicator_id'])


In [4]:
df

,Unnamed: 0,indicator_id,region_id,date,value,year,month,CRAM,IRAM,LRAM,MRAM,NRAM,SRAM
0,0,ZATT,394521,2020-06-30,247142.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
1,1,ZSFH,394521,2020-06-30,143910.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
2,2,ZALL,394521,2020-06-30,142059.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
3,3,ZCON,394521,2020-06-30,116574.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
4,4,ZABT,394521,2020-06-30,77253.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
303915,303915,Z5BR,395237,2022-09-30,742851.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0
303916,303916,Z2BR,395237,2022-09-30,334812.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0
303917,303917,Z3BR,395237,2022-09-30,435978.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0
303918,303918,Z1BR,395237,2022-09-30,388806.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0


In [9]:
# from sklearn.preprocessing import LabelEncoder

# # Create a LabelEncoder instance
# label_encoder = LabelEncoder()
# df = pd.read_csv('/Users/hm/Documents/Github_Projects/Zillow-data-ML-project/artifacts/data_ingestion/final.csv')

# # Fit and transform the 'indicator_id' column
df['encoded_indicator_id'] = label_encoder.fit_transform(df['indicator_id'])

# Create a mapping dictionary to store the label-to-category mapping
label_to_category_mapping = dict(zip(df['encoded_indicator_id'], df['indicator_id']))

# Print the mapping
print(label_to_category_mapping)


{7: 'ZATT', 9: 'ZSFH', 6: 'ZALL', 8: 'ZCON', 5: 'ZABT', 4: 'Z5BR', 1: 'Z2BR', 2: 'Z3BR', 0: 'Z1BR', 3: 'Z4BR'}


In [11]:
df

,region_id,date,value,year,month,CRAM,IRAM,LRAM,MRAM,NRAM,SRAM
0,394521,2020-06-30,247142.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
1,394521,2020-06-30,143910.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
2,394521,2020-06-30,142059.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
3,394521,2020-06-30,116574.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
4,394521,2020-06-30,77253.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0
...,...,...,...,...,...,...,...,...,...,...,...
303915,395237,2022-09-30,742851.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0
303916,395237,2022-09-30,334812.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0
303917,395237,2022-09-30,435978.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0
303918,395237,2022-09-30,388806.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0


In [5]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

label_encoder = LabelEncoder()
df['encoded_indicator_id'] = label_encoder.fit_transform(df['indicator_id'])
df.drop(['Unnamed: 0','indicator_id'], axis=1, inplace= True)

# Select relevant columns
columns_to_use = ['encoded_indicator_id', 'region_id', 'year', 'month', 'CRAM', 'IRAM', 'LRAM', 'MRAM', 'NRAM', 'SRAM']

# Define features and target variable
X = df[columns_to_use]
y = df['value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
df.drop(['Unnamed: 0','indicator_id'], axis=1)

,region_id,date,value,year,month,CRAM,IRAM,LRAM,MRAM,NRAM,SRAM,encoded_indicator_id
0,394521,2020-06-30,247142.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,7
1,394521,2020-06-30,143910.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,9
2,394521,2020-06-30,142059.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,6
3,394521,2020-06-30,116574.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,8
4,394521,2020-06-30,77253.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
303915,395237,2022-09-30,742851.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,4
303916,395237,2022-09-30,334812.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,1
303917,395237,2022-09-30,435978.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,2
303918,395237,2022-09-30,388806.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,0


In [25]:
!pip install dill

In [32]:
import os, pickle

def save_object_to_pickle(object, save_folder, file_name):
        try:
            # Create the full save path, including the "artifacts/" folder
            save_path = os.path.join("artifacts", save_folder, file_name)

            # Create the directory if it doesn't exist
            os.makedirs(os.path.dirname(save_path), exist_ok=True)

            # Save the trained model to a pickle file
            with open(save_path, 'wb') as file:
                pickle.dump(object, file)
            # keras_model.save(save_path)

            print(f"Model saved to {save_path}")
        except Exception as e:
            print(f"Error saving the model: {str(e)}")

def load_pickle_object(load_folder, file_name):
    try:
        # Create the full load path, including the "artifacts/" folder
        load_path = os.path.join("artifacts", load_folder, file_name)

        # Load the model from the pickle file
        with open(load_path, 'rb') as file:
            loaded_model = pickle.load(file)
        # loaded_model = keras.models.load_model(load_path)

        return loaded_model

    except Exception as e:
        print(f"Error loading the model: {str(e)}")

In [28]:
save_object_to_pickle( X_test_scaled, "models", "X_test_scaled.pkl")

Model saved to artifacts/models/X_test_scaled.pkl


In [2]:
load_pickle_object("models", "X_test_scaled.pkl")

NameError: name 'load_pickle_object' is not defined

In [6]:
df

,region_id,date,value,year,month,CRAM,IRAM,LRAM,MRAM,NRAM,SRAM,encoded_indicator_id
0,394521,2020-06-30,247142.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,7
1,394521,2020-06-30,143910.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,9
2,394521,2020-06-30,142059.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,6
3,394521,2020-06-30,116574.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,8
4,394521,2020-06-30,77253.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
303915,395237,2022-09-30,742851.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,4
303916,395237,2022-09-30,334812.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,1
303917,395237,2022-09-30,435978.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,2
303918,395237,2022-09-30,388806.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,0


In [8]:
X_train

,encoded_indicator_id,region_id,year,month,CRAM,IRAM,LRAM,MRAM,NRAM,SRAM
194244,3,395229,2021,4,0.108794,1063.0,349000.0,21.0,5.0,290000.0
74223,2,395238,2018,8,0.252730,3776.0,299000.0,35.0,16.0,265000.0
244707,5,394353,2022,2,0.076923,229.0,279900.0,49.0,10.0,225000.0
213079,5,395063,2021,8,0.137255,131.0,250000.0,18.0,6.0,229000.0
188507,4,394910,2021,3,0.163148,2839.0,285000.0,32.0,8.0,239700.0
...,...,...,...,...,...,...,...,...,...,...
119879,4,394492,2019,9,0.316820,7164.0,249899.0,30.0,10.0,197500.0
259178,0,395033,2022,5,0.131169,726.0,175000.0,17.0,5.0,167000.0
131932,7,394875,2019,12,0.101535,1796.0,189900.0,80.0,44.0,153750.0
146867,4,395030,2020,4,0.159334,774.0,274900.0,33.0,13.0,229500.0


In [10]:
df

,region_id,date,value,year,month,CRAM,IRAM,LRAM,MRAM,NRAM,SRAM,encoded_indicator_id
0,394521,2020-06-30,247142.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,7
1,394521,2020-06-30,143910.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,9
2,394521,2020-06-30,142059.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,6
3,394521,2020-06-30,116574.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,8
4,394521,2020-06-30,77253.0,2020,6,0.14970,2210.0,208990.0,25.0,4.0,158400.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
303915,395237,2022-09-30,742851.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,4
303916,395237,2022-09-30,334812.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,1
303917,395237,2022-09-30,435978.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,2
303918,395237,2022-09-30,388806.0,2022,9,0.18595,228.0,226450.0,18.0,7.0,334000.0,0


In [11]:
X_test

,encoded_indicator_id,region_id,year,month,CRAM,IRAM,LRAM,MRAM,NRAM,SRAM
278515,5,753869,2022,8,0.175799,441.0,194900.0,13.0,5.0,195000.0
39033,3,753869,2023,5,0.166667,283.0,237500.0,14.0,4.0,220000.0
253014,9,394308,2022,4,0.104285,1745.0,279777.0,21.0,6.0,286000.0
226918,7,394579,2021,11,0.200238,927.0,175000.0,22.0,8.0,174000.0
177475,5,394728,2020,12,0.093074,502.0,189900.0,58.0,26.0,167500.0
...,...,...,...,...,...,...,...,...,...,...
218758,2,753877,2021,9,0.132505,492.0,449000.0,53.0,24.0,325000.0
117867,6,394729,2019,8,0.197927,963.0,149900.0,49.0,31.0,115000.0
269960,8,394418,2022,6,0.238095,115.0,516950.0,67.0,14.0,475000.0
225870,6,394619,2021,11,0.122653,1814.0,397500.0,22.0,8.0,360000.0
